In [0]:
import torch
from torch import tensor, manual_seed, rand
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly import express as px

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

# this is for rounding torch tensors
decimal_places = 3 # round to ... decimal places
rounder = 10**decimal_places

# Matrix inverse

In [78]:
M1234 = torch.diag(tensor([1.,2.,3.,4.]))
M1234

tensor([[1., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 3., 0.],
        [0., 0., 0., 4.]])

In [0]:
assert torch.det(M1234), 'Should not be zero'

## Let's use the builtin method first

In [121]:
inverse = torch.inverse(M1234)
(inverse * rounder).round()/rounder


tensor([[1.0000, 0.0000, 0.0000, -0.0000],
        [0.0000, 0.5000, 0.0000, -0.0000],
        [0.0000, 0.0000, 0.3330, -0.0000],
        [0.0000, 0.0000, 0.0000, 0.2500]])

In [124]:
# Let's check it's really the inverse ... of course it is!
decimal_places = 3 # round to ... decimal places

isThisIdentity = inverse @ M1234
isThisIdentity.float()

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

# Now let's use gradient descent to do the same

So what we want a matrix that - multiplied by the original matrix M1234 - yields the identity matrix

In [126]:
identity = torch.diag(torch.tensor([1,1,1,1])).float()
Y_true = identity
Y_true.float()

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [0]:
manual_seed(314)
random_inverse = torch.rand(size=[4,4], requires_grad=True)
# random_inverse = torch.nn.Parameter(random_inverse) 

In [128]:
Y_hat = random_inverse @ M1234
Y_hat.float()

tensor([[0.7196, 1.2590, 2.0002, 1.3540],
        [0.8522, 0.6251, 1.5017, 1.8571],
        [0.0083, 0.8938, 2.4086, 2.2047],
        [0.3556, 0.7420, 2.0490, 1.2297]], grad_fn=<MmBackward>)

Of course random_inverse, is not yet the inverse, therefore Y_hat is not the identity matrix. 


---

A useful distance is the element-wise mse

In [0]:
def mse(y, y_hat): return ((y-y_hat)**2).mean()

In [0]:
loss = mse(Y_true, Y_hat)

# let's calculate the gradients
loss.backward()

In [133]:
# random_inverse.grad is the gradient at each position of the matrix
# to get a better understanding let's just take a look
random_inverse.grad

tensor([[-3.5046e-02,  3.1475e-01,  7.5007e-01,  6.7699e-01],
        [ 1.0653e-01, -9.3718e-02,  5.6315e-01,  9.2856e-01],
        [ 1.0338e-03,  2.2346e-01,  5.2824e-01,  1.1024e+00],
        [ 4.4448e-02,  1.8549e-01,  7.6836e-01,  1.1486e-01]])

# Let us use gradient descent with respect to the random_inverse and mse as the loss function to approximate the inverse matrix.

## Set the learning rate for gradient descent. The learning rate is one of the most important paramters controlling the gradient descent method. 
One common way of visualising gradient descent compares the loss function to some sort of hilly landscape. The current combination of parameters (in this case eg. our random inverse matrix) corresponds to a position in this landscape. We would like to get to a valley in this landscape. One way is to put a "ball" at the current position and let it roll. This is a fine procedure, however we want one that is computationally simpler. Therefore we don't allow the ball to roll and gather momentum etc, but we just displace in the downhill direction by some distance proportional to the current slope. The proportionality constant is what is called the learning rate.

In [0]:
learning_rate = 0.9

In [0]:
def update(M, lr, losses, momentum, plot):
  Y_hat = random_inverse@M
  loss = mse(Y_true, Y_hat)
  loss.backward()
  losses.append(loss.detach().numpy())


  if t%2000==0 and plot:
    print(t,':', loss)
    Y_hat = random_inverse @ M

    # plot the intermediate results
    G_true = Y_true.detach().numpy()
    G_hat = Y_hat.detach().numpy()
    Diff = G_hat - G_true

    fig = px.imshow(Diff,range_color=[-1,1])
    fig.update_yaxes(showticklabels=False)
    fig.update_layout(title=f'<b>Difference: Approximate minus True Identity.</b><br>Iteration:{t}.<br>Loss:{loss:.2}')
    fig.show()
    print(Diff)

  with torch.no_grad():
    random_inverse.sub_(lr * random_inverse.grad)

    # random_inverse.grad is the gradient at each position of the matrix, it might look like this: 
    #tensor([[0.3002, 0.3010, 0.4035, 0.0731],
    #    [0.2227, 0.2393, 0.2634, 0.0491],
    #    [0.1516, 0.1076, 0.2110, 0.0296],
    #    [0.2051, 0.1801, 0.2372, 0.0551]])

    random_inverse.grad *= momentum

    return None


In [0]:
M1234 = torch.diag(tensor([1.,2.,3.,4.]))
M_random = torch.rand(size=(4,4))

In [106]:
losses1 = []

manual_seed(314)
random_inverse = torch.rand(size=[4,4])
random_inverse = torch.nn.Parameter(random_inverse) 
for t in range(10000): update(M1234.clone(), learning_rate, losses1, momentum=0.9, plot=True)


0 : tensor(1.6650, grad_fn=<MeanBackward0>)


[[-0.28 1.26 2.00 1.35]
 [0.85 -0.37 1.50 1.86]
 [0.01 0.89 1.41 2.20]
 [0.36 0.74 2.05 0.23]]
2000 : tensor(1.9984e-15, grad_fn=<MeanBackward0>)


[[0.00 -0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00]
 [0.00 -0.00 -0.00 0.00]
 [0.00 -0.00 0.00 -0.00]]
4000 : tensor(1.1102e-15, grad_fn=<MeanBackward0>)


[[0.00 -0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00]
 [0.00 -0.00 -0.00 0.00]
 [0.00 -0.00 0.00 -0.00]]
6000 : tensor(1.9984e-15, grad_fn=<MeanBackward0>)


[[0.00 -0.00 -0.00 0.00]
 [0.00 -0.00 -0.00 0.00]
 [0.00 -0.00 -0.00 0.00]
 [0.00 -0.00 -0.00 -0.00]]
8000 : tensor(1.3323e-15, grad_fn=<MeanBackward0>)


[[0.00 -0.00 0.00 0.00]
 [-0.00 -0.00 0.00 0.00]
 [-0.00 -0.00 -0.00 0.00]
 [-0.00 -0.00 0.00 -0.00]]


In [116]:
losses2 = []
M_random = torch.rand(size=(4,4))

manual_seed(314)
random_inverse = torch.rand(size=[4,4])
random_inverse = torch.nn.Parameter(random_inverse) 
for t in range(10000): update(M_random.clone(), learning_rate, losses2, 0.9, plot=True)

0 : tensor(0.8493, grad_fn=<MeanBackward0>)


[[-0.36 1.25 1.76 1.08]
 [0.59 0.12 1.45 0.87]
 [0.37 0.93 0.21 0.80]
 [0.45 0.97 1.25 -0.18]]
2000 : tensor(2.6224e-12, grad_fn=<MeanBackward0>)


[[-0.00 0.00 0.00 -0.00]
 [0.00 -0.00 -0.00 0.00]
 [0.00 -0.00 -0.00 0.00]
 [-0.00 0.00 0.00 -0.00]]
4000 : tensor(2.6580e-12, grad_fn=<MeanBackward0>)


[[-0.00 0.00 0.00 -0.00]
 [0.00 -0.00 -0.00 0.00]
 [0.00 -0.00 -0.00 0.00]
 [-0.00 0.00 0.00 -0.00]]
6000 : tensor(2.6580e-12, grad_fn=<MeanBackward0>)


[[-0.00 0.00 0.00 -0.00]
 [0.00 -0.00 -0.00 0.00]
 [0.00 -0.00 -0.00 0.00]
 [-0.00 0.00 0.00 -0.00]]
8000 : tensor(2.6580e-12, grad_fn=<MeanBackward0>)


[[-0.00 0.00 0.00 -0.00]
 [0.00 -0.00 -0.00 0.00]
 [0.00 -0.00 -0.00 0.00]
 [-0.00 0.00 0.00 -0.00]]


In [0]:
losses3 = []

manual_seed(314)
random_inverse = torch.rand(size=[4,4])
random_inverse = torch.nn.Parameter(random_inverse) 
for t in range(10000): update(M1234.clone(), learning_rate, losses3, 0.1, plot=False)

In [0]:
losses4 = []

manual_seed(314)
random_inverse = torch.rand(size=[4,4])
random_inverse = torch.nn.Parameter(random_inverse) 
for t in range(10000): update(M_random.clone(), learning_rate, losses4, 0.1, plot=False)

Compare convergence speed for simple diagonal matrix and general random matrix

In [119]:
import pandas as pd

import plotly.express as px
df = pd.DataFrame(
    {
     'losses':np.array(losses1).flatten(),
     'losses2':np.array(losses2).flatten(),
     'losses3':np.array(losses3).flatten(),
     'losses4':np.array(losses4).flatten()
     })
fig = px.line(df, y='losses', log_y=True)
fig.update_traces(name='Simple Diagonal Matrix, Momentum=0.9', showlegend = True)
fig.add_scatter(y=df['losses2'], name='Difficult Random Matrix,  Momentum=0.9')
fig.add_scatter(y=df['losses3'], name='Simple Diagonal Matrix, Momentum=0.0')
fig.add_scatter(y=df['losses4'], name='Difficult Random Matrix, Momentum=0.0')
fig.show()